## Black Scholes model

*This exploratory notebook is a base for an option pricer app with visualizations and other features.*

The Black Scholes equation is a PDE that governs the price of a derivative.

---

$$\frac{\partial C}{\partial t}+\frac{1}{2}\sigma^2S^2\frac{\partial^2 C}{\partial S^2} + rS\frac{\partial C}{\partial S}-rC=0$$

With C the price of some derivative.

---

6 variables:

- $S$: Underlying asset current price.
- $K$: Strike price of the option.
- $T$: Time to maturity in years.
- $r$: Risk-free interest rate.
- $\sigma$: Volatility of the underlying (standard deviation).
- $q$: Dividend yield.

Call option value:

$$C=Se^{-qT}N(d_1)-Ke^{-rT}N(d_2)$$

Put option value:

$$P=Ke^{-rT}N(-d_2)-Se^{-qT}N(-d_1)$$

---

$$d_1=\frac{\mathrm{ln}(\frac{S}{K})+(r-q+0.5\sigma^2)T}{\sigma \sqrt{T}}$$

$$d_2=d_1-\sigma \sqrt{T}$$



In [3]:
import jax.numpy as jnp
from jax import grad
from jax.scipy.stats import norm as jnorm
def blackScholes(r, S, K, T, sigma, q=0,type="C"):
    "Calculate BSM option price for a call/put"
    d1 = (jnp.log(S/K) + (r-q+0.5*sigma**2)*T)/(sigma*jnp.sqrt(T))
    d2 = d1 - sigma*jnp.sqrt(T)
    try:
        if type=="C":
            price=S*jnp.exp(-q*T)*jnorm.cdf(d1, 0, 1) - K*jnp.exp(-r*T)*jnorm.cdf(d2, 0, 1)
        elif type=="P":
            price=K*jnp.exp(-r*T)*jnorm.cdf(-d2, 0, 1) - S*jnp.exp(-q*T)*jnorm.cdf(-d1, 0, 1)
        return price
    except:
        print("Please confirm all option parameters.")


In [8]:
S=100.0
K=110.0
T=0.8
r=0.05
sigma=0.2
q=0.0

print("Option price is:", round(blackScholes(r, S, K, T, sigma, type="C"), 2))

Option price is: 4.83


## Greeks:

- $\Delta=\frac{\partial C}{\partial S}$ represents the variation of the option price when the underlying price changes. (1)

- $\Gamma =\frac{\partial^2C}{\partial S^2}=\frac{\partial \Delta}{\partial S}$ represents the variation of the option price when the volatility changes. (2)

- $\Theta=\frac{\partial C}{\partial T}$ represents the variation of the option price when the time to maturity changes. (3)

- $\rho=\frac{\partial C}{\partial \sigma}$ represents the variation of the option price when the interest rate changes. (4)

- $\nu = \frac{\partial C}{\partial \sigma}$ represents the variation of the option price when the volatility changes. (5)

In [ ]:
# Computing of the greeks

jax_delta = grad(blackScholes, argnums=1)

jax_gamma = grad(jax_delta, argnums=1)

jax_vega = grad(blackScholes, argnums=4)

jax_rho = grad(blackScholes, argnums=0)

jax_theta = grad(blackScholes, argnums=3)

In [12]:
delta = jax_delta(r, S, K, T, sigma, q=0,type="C")
gamma = jax_gamma(r, S, K, T, sigma, q=0,type="C")
vega = jax_vega(r, S, K, T, sigma, q=0,type="C")
rho = jax_rho(r, S, K, T, sigma, q=0,type="C")
theta = -jax_theta(r, S, K, T, sigma, q=0,type="C")
print(delta, gamma, vega, rho, theta)

0.4130328 0.021769531 34.83124 29.176401 -6.17743


## Implied volatility 

Putting a clear value on the volatility of an asset is not as simple as knowing the value of the other variables involved in the black scholes model.

---

The baseline is that the *market value of the option implies a value for the volatility*.

In [14]:
def loss(r, S, K, T, sigma_guess, price, q=0, type="C"):
    
    # Price with a guess for vol
    theoretical_price = blackScholes(r, S, K, T, sigma_guess, q, type)
    # Price the option is selling at
    market_price = price

    # Loss is the difference between the theoretical price and the actual price
    # The goal is to minimize the loss function
    return theoretical_price - market_price
loss_grad = grad(loss, argnums=4)

We can use the Newton Raphson numerical method.

$x^g$ is the guess for vol.

$x_{n+1}^g=x_n^g-\frac{f(x_n^g)}{f'(x_n^g)}$

$f(x_n^g)$ represents the loss function. The difference between:

- $P_{theory}$: Theoretical price using the volatility guess.

- $P_{market}$: Market price.

$f'(x_n^g)$ is the gradient of the loss function.

$$\sigma_{n+1}=\sigma_n-\frac{\mathcal{L}(\sigma_n)}{\frac{\partial\mathcal{L}}{\partial\sigma}}$$

Where $\mathcal{L}$ represents the loss function.